# LightGBM Quantile Regression - Global Model

**ОБНОВЛЕНО**: Этот ноутбук теперь использует **глобальную модель** на всех тикерах!

## Основные изменения:
- ✅ Агрегация данных всех тикеров из `data/processed_ml/`
- ✅ Строгий **временной split** (без shuffle!) - cutoff 2024-01-01
- ✅ Квантильные модели: 16%, 50%, 84% (±1σ)
- ✅ Sample weighting по ликвидности
- ✅ Категориальные признаки: `ticker_id`, `sector_id`

## Файлы:
- **Скрипт**: `train_global_model.py` (полный pipeline)
- **Модели**: `data/models/global_lgbm_q{16,50,84}.txt`
- **Отчёты**: `reports/feature_importance.png`

Для запуска полного pipeline используйте скрипт:
```bash
python 03_models/train_global_model.py
```


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Добавляем путь к корневой директории ML
sys.path.insert(0, str(Path.cwd().parent))

# Попытка импорта lightgbm
try:
    import lightgbm as lgb
    print("✅ LightGBM доступен")
    LGB_AVAILABLE = True
except ImportError:
    print("⚠️ Библиотека 'lightgbm' не установлена. Установите: pip install lightgbm")
    LGB_AVAILABLE = False

# Импорт модуля обучения
try:
    from train_global_model import (
        Config,
        load_all_ticker_data,
        create_target_variable,
        time_series_split,
        prepare_lgbm_data,
        train_quantile_models,
        quantile_loss
    )
    print("✅ Модуль train_global_model импортирован")
except ImportError as e:
    print(f"⚠️ Не удалось импортировать train_global_model: {e}")
    print("   Используйте скрипт напрямую: python train_global_model.py")


## Загрузка и подготовка данных


In [ ]:
DATA_DIR = Path('data') / 'features'
ticker = 'SBER'
df = pd.read_parquet(DATA_DIR / f"{ticker}_with_targets.parquet")

# Выбираем признаки для модели (исключаем целевые и вспомогательные)
feature_cols = [col for col in df.columns if not col.startswith('target_') and 
                not col.startswith('quantile_') and 
                col not in ['date', 'ticker', 'begin', 'end']]

# Целевая переменная - 5-дневная волатильность
target_col = 'target_vol_5d'

# Удаляем строки с NaN
df_clean = df[feature_cols + [target_col]].dropna()

X = df_clean[feature_cols]
y = df_clean[target_col]

print(f"✅ Данные подготовлены")
print(f"Признаков: {len(feature_cols)}")
print(f"Наблюдений: {len(X)}")
print(f"\nПризнаки: {feature_cols[:10]}...")


## Обучение квантильных моделей


In [ ]:
if LGB_AVAILABLE:
    # Разделение на train/test (последние 20% - тест)
    split_idx = int(len(X) * 0.8)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
    
    # Обучение моделей для разных квантилей
    quantiles = [0.16, 0.50, 0.84]  # ±1σ и медиана
    models = {}
    predictions = {}
    
    for q in quantiles:
        print(f"\nОбучение модели для квантиля {q:.2f}...")
        
        params = {
            'objective': 'quantile',
            'alpha': q,
            'metric': 'quantile',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': -1
        }
        
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
        
        model = lgb.train(
            params,
            train_data,
            num_boost_round=200,
            valid_sets=[valid_data],
            callbacks=[lgb.early_stopping(stopping_rounds=20), lgb.log_evaluation(period=0)]
        )
        
        models[q] = model
        predictions[f'pred_q{int(q*100)}'] = model.predict(X_test)
        
        print(f"✅ Модель для квантиля {q:.2f} обучена")
    
    # Создаем DataFrame с прогнозами
    results_df = pd.DataFrame(predictions)
    results_df['actual'] = y_test.values
    
    print(f"\n✅ Все модели обучены")
    print(f"\nПример прогнозов:")
    print(results_df.head())
    
else:
    print("❌ LightGBM недоступен")


## Оценка качества


In [ ]:
if LGB_AVAILABLE:
    # Проверка покрытия квантилей
    coverage_16 = (results_df['actual'] <= results_df['pred_q16']).mean()
    coverage_84 = (results_df['actual'] <= results_df['pred_q84']).mean()
    
    # MAE для медианы
    mae_median = np.mean(np.abs(results_df['actual'] - results_df['pred_q50']))
    
    print(f"\n=== Метрики LightGBM Quantile ===")
    print(f"Покрытие 16% квантиля: {coverage_16:.2%} (ожидается ~16%)")
    print(f"Покрытие 84% квантиля: {coverage_84:.2%} (ожидается ~84%)")
    print(f"MAE для медианы: {mae_median:.4f}")
    
    # Feature importance
    print(f"\n=== Важность признаков (для 50% квантиля) ===")
    importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': models[0.50].feature_importance()
    }).sort_values('importance', ascending=False)
    
    print(importance.head(10))
    
else:
    print("❌ Оценка недоступна")


## Сохранение результатов


In [ ]:
if LGB_AVAILABLE:
    OUTPUT_DIR = Path('data') / 'models'
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    # Сохранение прогнозов
    output_path = OUTPUT_DIR / f"{ticker}_lgb_quantile_forecasts.parquet"
    results_df.to_parquet(output_path, index=False)
    
    # Сохранение важности признаков
    importance_path = OUTPUT_DIR / f"{ticker}_lgb_feature_importance.csv"
    importance.to_csv(importance_path, index=False)
    
    print(f"✅ Прогнозы сохранены: {output_path}")
    print(f"✅ Важность признаков: {importance_path}")
    
else:
    print("⚠️ Нечего сохранять")
